In [3]:
from urllib.request import urlopen
import re # Regex Library
import pandas as pd

In [4]:
def get_html_from_url(url): 
    return urlopen(url).read().decode("utf-8")

In [5]:
states = ['Alabama',
 'Alaska',
 'Arizona',
 'Arkansas',
 'California',
 'Colorado',
 'Connecticut',
 'Delaware',
 'Florida',
 'Georgia',
 'Hawaii',
 'Idaho',
 'Illinois',
 'Indiana',
 'Iowa',
 'Kansas',
 'Kentucky',
 'Louisiana',
 'Maine',
 'Maryland',
 'Massachusetts',
 'Michigan',
 'Minnesota',
 'Mississippi',
 'Missouri',
 'Montana',
 'Nebraska',
 'Nevada',
 'New_Hampshire',
 'New_Jersey',
 'New_Mexico',
 'New_York',
 'North_Carolina',
 'North_Dakota',
 'Ohio',
 'Oklahoma',
 'Oregon',
 'Pennsylvania',
 'Rhode_Island',
 'South_Carolina',
 'South_Dakota',
 'Tennessee',
 'Texas',
 'Utah',
 'Vermont',
 'Virginia',
 'Washington',
 'West_Virginia',
 'Wisconsin',
 'Wyoming']

In [6]:
df = pd.read_csv("midterm_predictions.csv")
df.head()

,Unnamed: 0,district,state,chamber,winner_predicted_by_total,winner_predicted_by_unique,margin_total,margin_unique,actual_winner,affiliation
0,0,District 1,Alabama,house,Robert Kennedy Jr.,Robert Kennedy Jr.,0.280000,0.090909,Bradley Byrne,Republican Party
1,122,District 2,Alabama,house,Tabitha Isner,Tabitha Isner,0.932584,0.967742,Martha Roby,Republican Party
2,196,District 3,Alabama,house,Mallory Hagan,Mallory Hagan,NaN,NaN,Mike Rogers,Republican Party
3,248,District 4,Alabama,house,Lee Auman,Lee Auman,1.000000,1.000000,Robert Aderholt,Republican Party
4,290,District 5,Alabama,house,Peter Joffrion,Peter Joffrion,0.726872,0.768116,Mo Brooks,Republican Party


In [7]:
from bs4 import BeautifulSoup

In [8]:
def clean_winner_name(winner_name):
    words_list = winner_name.text.split(' ')
    final_name = []
    for w in words_list:
        if not(w == ''):
            final_name.append(w.strip('\n'))
    return final_name

# find winners from parsing for republican, democratic or no party preference winners
def find_winners(soup):
    table_boxes = soup.find_all('table')
    winner_boxes = []
    for table in table_boxes:
        winner_box = table.find('tr', attrs={'class':[' republican winner', ' democratic winner', ' no-party-preference winner']})
        winner_boxes.append(winner_box)
    winner_names = []
    for winner_box in winner_boxes:
        if winner_box:
            affiliation = ''
            if 'republican' in winner_box.attrs['class']:
                affiliation = 'republican'
            if 'democratic' in winner_box.attrs['class']:
                affiliation = 'democratic'
            if 'no-party-preference' in winner_box.attrs['class']:
                affiliation = 'no party preference'
            winner_name = winner_box.find('td', attrs={'class': 'name-cell'})
            winner_name_clean = ' '.join(clean_winner_name(winner_name))
            winner_names.append((winner_name_clean, affiliation))
        else:
            winner_names.append(winner_box)
    return winner_names

# parse the state from the title in the webpage
def get_state(title):
    title_list = title.split(' ')
    if title_list[1] != 'US':
        return title_list[0] + '_' + title_list[1]
    else:
        return title_list[0]
    
def update(soup, winner_names):
    # find state and district of each winner
    divs = soup.find_all('div', attrs={'class':'text-center'})
    for i in range(len(divs)):
        title = divs[i].find('h2').text
        state = get_state(title)
        try:
            district_num = int(title.split(' ')[-1])
        except:
            district_num = None
        if district_num:
            district = 'District ' + str(district_num)
            try:
                name, affiliation = winner_names[i] 
                print(name, affiliation, district, state)
                # update dataframe
                df.loc[(df['state'] == state) & (df['district'] == district), 'actual_winner'] = name 
                df.loc[(df['state'] == state) & (df['district'] == district), 'affiliation'] = affiliation
            except:
                pass
    return df
        
def update_one_state(state):
    state_url_name = state.replace('_', '-')
    page = urlopen('https://www.vox.com/a/midterms-2018/{}-election-results'.format(state_url_name.lower()))
    # parse the html using beautiful soup and store in variable `soup`
    soup = BeautifulSoup(page, 'html.parser')
    winner_names = find_winners(soup)
    df = update(soup, winner_names)
    return df

def update_all(states):
    for state in states:
        update_one_state(state)
    


In [9]:
update_all(states)

 Bradley Byrne republican District 1 Alabama
 Martha Roby republican District 2 Alabama
 Mike Rogers republican District 3 Alabama
 Robert Aderholt republican District 4 Alabama
 Mo Brooks republican District 5 Alabama
 Gary Palmer republican District 6 Alabama
 Terri Sewell democratic District 7 Alabama
  Affirmative no party preference District 1 Alabama_State
 Don Young republican District 1 Alaska
 Tom O'Halleran democratic District 1 Arizona
 Ann Kirkpatrick democratic District 2 Arizona
 Raul Grijalva democratic District 3 Arizona
 Paul Gosar republican District 4 Arizona
 Andy Biggs republican District 5 Arizona
 David Schweikert republican District 6 Arizona
 Ruben Gallego democratic District 7 Arizona
 Debbie Lesko republican District 8 Arizona
 Greg Stanton democratic District 9 Arizona
  Affirmative no party preference District 126 Arizona_State
 Rick Crawford republican District 1 Arkansas
 French Hill republican District 2 Arkansas
 Steve Womack republican District 3 Arkan

 Steve Scalise republican District 1 Louisiana
 Cedric Richmond democratic District 2 Louisiana
 Clay Higgins republican District 3 Louisiana
 Mike Johnson republican District 4 Louisiana
 Ralph Abraham republican District 5 Louisiana
 Garret Graves republican District 6 Louisiana
  Affirmative no party preference District 1 Louisiana_State
 Chellie M Pingree democratic District 1 Maine
  No no party preference District 1 Maine_State
 Andrew P Harris republican District 1 Maryland
 C. A. Dutch Ruppersberger democratic District 2 Maryland
 John Sarbanes democratic District 3 Maryland
 Anthony G. Brown democratic District 4 Maryland
 Steny  Hoyer democratic District 5 Maryland
 David Trone democratic District 6 Maryland
 Elijah Cummings democratic District 7 Maryland
 Jamie Raskin democratic District 8 Maryland
  Affirmative no party preference District 2 Maryland_State
 Richard Neal democratic District 1 Massachusetts
 James P McGovern democratic District 2 Massachusetts
 Lori Trahan de

 David N Cicilline democratic District 1 Rhode_Island
 James R. Langevin democratic District 2 Rhode_Island
 Joe Cunningham democratic District 1 South_Carolina
 Joe  Wilson republican District 2 South_Carolina
 Jeff Duncan republican District 3 South_Carolina
 William Timmons republican District 4 South_Carolina
 Ralph Norman republican District 5 South_Carolina
 James  Clyburn democratic District 6 South_Carolina
 Tom Rice republican District 7 South_Carolina
 Dustin "Dusty" Johnson republican District 1 South_Dakota
 Phil Roe republican District 1 Tennessee
 Tim Burchett republican District 2 Tennessee
 Chuck Fleischmann republican District 3 Tennessee
 Scott DesJarlais republican District 4 Tennessee
 Jim Cooper democratic District 5 Tennessee
 John Rose republican District 6 Tennessee
 Mark Green republican District 7 Tennessee
 David Kustoff republican District 8 Tennessee
 Steve Cohen democratic District 9 Tennessee
 Louie Gohmert republican District 1 Texas
 Dan Crenshaw republ

In [10]:
df.to_csv("midterm_results.csv")